### Agent-Lab: Multi-Agent -> Browser

Objective of this notebook is evaluating and adapting a [Multi-Agent Supervisor Architecture](https://langchain-ai.github.io/langgraph/concepts/multi_agent/#supervisor) with coordinator and execution planning steps.

---


In [1]:
%%capture

import json
import os

os.chdir("..")

from IPython.display import Markdown, display

from dotenv import load_dotenv

load_dotenv()

from notebooks import experiment_utils
from app.core.container import Container
from app.interface.api.messages.schema import MessageRequest

import nest_asyncio

nest_asyncio.apply()

# start dependency injection container
container = Container()
container.init_resources()
container.wire(modules=[__name__])

# get checkpointer instance
graph_persistence_factory = container.graph_persistence_factory()
checkpointer = graph_persistence_factory.build_checkpoint_saver()

---
### Anthropic Browser

In [2]:
# Create Workflow
anthropic_agent = experiment_utils.create_anthropic_agent(
    agent_type="coordinator_planner_supervisor", llm_tag="claude-sonnet-4-0", api_key=os.getenv("ANTHROPIC_API_KEY")
)
anthropic_browser_agent = container.coordinator_planner_supervisor_agent()
anthropic_workflow_builder = anthropic_browser_agent.get_workflow_builder(anthropic_agent["id"])
anthropic_workflow = anthropic_workflow_builder.compile(checkpointer=checkpointer)

In [3]:
%%capture

message = MessageRequest(
    message_role="human",
    message_content="Visit https://en.wikipedia.org/wiki/Mathematical_finance and rewrite the first paragraph in a simpler way that a 12 year old would understand.",
    agent_id=anthropic_agent["id"],
)

inputs = anthropic_browser_agent.get_input_params(message, schema="public")
config = anthropic_browser_agent.get_config(anthropic_agent["id"])
result = anthropic_workflow.invoke(inputs, config)
ai_message_content, workflow_state = anthropic_browser_agent.format_response(result)

In [4]:
display(
    Markdown(
        f"**Execution Plan:**\n```json\n{json.dumps(workflow_state.get("execution_plan"), indent=2)}\n```"
    )
)

**Execution Plan:**
```json
{
  "thought": "The user wants me to visit a specific Wikipedia page about Mathematical finance and then rewrite the first paragraph in a way that a 12-year-old would understand. This involves two main tasks: 1) Using the browser agent to visit the Wikipedia page and extract the first paragraph, and 2) Using the reporter agent to rewrite that content in simpler language suitable for a 12-year-old audience.",
  "title": "Simplify Mathematical Finance Wikipedia Article for 12-Year-Olds",
  "steps": [
    {
      "agent_name": "browser",
      "title": "Extract First Paragraph from Mathematical Finance Wikipedia Page",
      "description": "Visit https://en.wikipedia.org/wiki/Mathematical_finance and extract the first paragraph of the article. The browser agent should navigate to the page, locate the first paragraph of content, and provide the exact text for rewriting."
    },
    {
      "agent_name": "reporter",
      "title": "Rewrite Content for 12-Year-Old Understanding",
      "description": "Take the first paragraph extracted from the Wikipedia page and rewrite it in simple, clear language that a 12-year-old would easily understand. Use everyday words, shorter sentences, and relatable examples while maintaining the core meaning of the original content."
    }
  ]
}
```

In [5]:
display(Markdown(f"**AI Message Content:**\n\n{ai_message_content}"))

**AI Message Content:**

Now let me rewrite this paragraph in a way that a 12-year-old would understand:

**Original paragraph:**
"Mathematical finance, also known as quantitative finance and financial mathematics, is a field of applied mathematics, concerned with mathematical modeling in the financial field."

**Simplified version for a 12-year-old:**

Mathematical finance is like using math to understand money and how it works. It has other names too, like "quantitative finance" (which just means using numbers and math to study finance) and "financial mathematics." 

Basically, it's when really smart people use math equations and formulas to figure out things like:
- How much money you might make or lose when you invest
- How much things will cost in the future
- How banks and companies should handle their money

Think of it like this: if regular math helps you figure out how much change you'll get at the store, mathematical finance uses much more complicated math to help grown-ups make smart decisions about really big amounts of money - like when banks decide who to lend money to, or when companies try to predict if their stock prices will go up or down.

It's basically using math as a tool to solve money puzzles!

---
### OpenAI Browser

In [19]:
# Create Workflow
openai_agent = experiment_utils.create_openai_agent(
    agent_type="coordinator_planner_supervisor", llm_tag="gpt-5-nano", api_key=os.getenv("OPENAI_API_KEY")
)
openai_browser_agent = container.coordinator_planner_supervisor_agent()
openai_workflow_builder = openai_browser_agent.get_workflow_builder(openai_agent["id"])
openai_workflow = openai_workflow_builder.compile(checkpointer=checkpointer)

In [20]:
%%capture

message = MessageRequest(
    message_role="human",
    message_content="Visit https://en.wikipedia.org/wiki/Mathematical_finance write an executive summary from the first paragraph in a simpler way that a 12 year old would understand.",
    agent_id=openai_agent["id"],
)

inputs = openai_browser_agent.get_input_params(message, schema="public")
config = openai_browser_agent.get_config(openai_agent["id"])
result = openai_workflow.invoke(inputs, config)
ai_message_content, workflow_state = openai_browser_agent.format_response(result)

In [21]:
display(
    Markdown(
        f"**Execution Plan:**\n```json\n{json.dumps(workflow_state.get("execution_plan"), indent=2)}\n```"
    )
)

**Execution Plan:**
```json
{
  "thought": "I will restate the user's requirement, plan steps with assigned agents, and ensure math tasks go to the coder when needed. The goal is to fetch the first paragraph from the Wikipedia page on Mathematical finance, then distill it into a simple executive summary suitable for a 12-year-old, and finally present a professional report.",
  "title": "Executive summary of Mathematical finance (first paragraph) simplified for a 12-year-old",
  "steps": [
    {
      "agent_name": "browser",
      "title": "Fetch first paragraph from Wikipedia",
      "description": "Visit https://en.wikipedia.org/wiki/Mathematical_finance and extract the first paragraph of the article.",
      "note": "Direct interaction with the web page; avoid quoting verbatim in final summary."
    },
    {
      "agent_name": "researcher",
      "title": "Summarize for a 12-year-old",
      "description": "Take the first paragraph text from the browser step and rewrite it in simple, kid-friendly terms. Emphasize that math is used to understand money and markets, pricing, and risk.",
      "note": "No heavy math; keep language accessible to a 12-year-old."
    },
    {
      "agent_name": "reporter",
      "title": "Prepare final executive report",
      "description": "Produce a concise executive report presenting the simplified summary intended for a general audience and a 12-year-old reader.",
      "note": "Format as a short executive summary suitable for decision-makers and educators alike."
    }
  ]
}
```

In [22]:
display(Markdown(f"**AI Message Content:**\n\n{ai_message_content}"))

**AI Message Content:**

Executive summary
- Mathematical finance is a branch of applied math that uses math to study money and financial markets. It highlights two main areas that use advanced math: pricing financial contracts that depend on other assets (derivatives) and managing risk and investments (portfolio management). It is also known as quantitative finance and financial mathematics.

Key findings
- Also known as: quantitative finance and financial mathematics.
- Field type: a branch of applied mathematics.
- Core activity: mathematical modeling in finance.
- Two main areas requiring advanced math:
  - Derivatives pricing (how much complex financial contracts should cost).
  - Risk and portfolio management (assessing and managing investment risk).

Detailed analysis
- Simplified restatement for a younger audience:
  - Mathematical finance uses math to understand money and markets.
  - It focuses on two big problems that need strong math: figuring out the price of contracts based on assets (like options) and deciding how to handle risk and build investment portfolios.

Conclusions and recommendations
- This summary captures the first paragraph in plain language suitable for a 12-year-old.
- For a fuller understanding, additional sections of the article should be consulted; information beyond the first paragraph was not provided here.
- Source: Wikipedia, Mathematical finance (first paragraph).